In [1]:
import json
import re
import requests
import sys
import io
import pandas as pd


## Import Neo4j

auth =(username, password)

In [2]:
from neo4j import GraphDatabase
driver=GraphDatabase.driver(uri="bolt://localhost:7687",auth=("neo4j","a"))
session=driver.session()

## Image Data Resource

    This section uses the input file "screenid" that contains a list of screen IDs to get annotations from https://idr.openmicroscopy.org/

In [4]:
key1 = [] # contains the key
value1 = [] # contains the value
total = [] # holds the output


def idr(inputfile):
    IDR_BASE_URL = "https://idr.openmicroscopy.org"
    INDEX_PAGE = "%s/webclient/?experimenter=-1" % IDR_BASE_URL
    # create http session
    with requests.Session() as session:
        request = requests.Request('GET', INDEX_PAGE)
        prepped = session.prepare_request(request)
        response = session.send(prepped)
        if response.status_code != 200:
            response.raise_for_status()

    MAP_URL = "{base}/webclient/api/annotations/?type=map&{type}={screen_id}"

    with io.open(inputfile, 'r', encoding="utf-8") as file_input:
        for i in file_input.readlines():
            qs = {'base': IDR_BASE_URL, 'type': 'screen', 'screen_id': i}
            url = MAP_URL.format(**qs)
            for a in session.get(url).json()['annotations']:
                #print("****************************************************")
                namespace = a['ns']
                for v in a['values']:
                    key = v[0]
                    value = v[1]
                    r = re.sub(r'http\S+$', "", value)
                    a=r.strip()
                    key1.append(key)
                    value1.append(a)
            x ={k: b for k, b in zip(key1, value1)} 
            
            x["Screen_Id"] = i.strip('\n')
            
            
            total.append(x)
            
     # Writing to idroutput.json
        with open("idrOutput.json", "w") as outfile:

            outfile.write(json.dumps(total))
            print("done")
            
            
          
                                                       
            



                    
                    
                           
idr("screenid.pmid")

done


### Create dataframe from the output

In [5]:
df = pd.DataFrame.from_dict(total)
df

,Sample Type,Organism,Study Type,Screen Type,Screen Technology Type,Imaging Method,Publication Title,Publication Authors,PubMed ID,PMC ID,...,Data DOI,BioStudies Accession,Screen C,Screen D,Overview,Screen E,Study Notebook,Study Notebook URL,Experiment B,Experiment C
0,cell,Schizosaccharomyces pombe,high content screen,primary screen,gene deletion screen,spinning disk confocal microscopy,A genomic Multiprocess survey of machineries t...,"Graml V, Studera X, Lawson JLD, Chessel A, Gey...",25373780,PMC4648281,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,cell,Saccharomyces cerevisiae,high content screen,primary screen,protein screen,fluorescence microscopy,A novel single-cell screening platform reveals...,"Breker M, Gymrek M, Schuldiner M",23509072,PMC3601363,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,cell,Homo sapiens,high content screen,primary screen,RNAi screen,fluorescence microscopy,Integration of biological data by kernels on g...,"Hériché JK, Lees JG, Morilla I, Walter T, Petr...",24943848,PMC4142622,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,cell,Drosophila melanogaster,high content screen,primary screen,RNAi screen,fluorescence microscopy,Comparative RNAi screening identifies a conser...,"Rohn JL, Sims D, Liu T, Fedorova M, Schöck F, ...",21893601,PMC3171124,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,cell,Saccharomyces cerevisiae,high content screen,primary screen,protein screen,confocal microscopy,Global analysis of SUMO chain function reveals...,"Srikumar T, Lewicki MC, Costanzo M, Tkach JM, ...",23547032,PMC3613684,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,cell,Saccharomyces cerevisiae,high content screen,primary screen,gene deletion screen,fluorescence microscopy,Genome-wide analysis of Rad52 foci reveals div...,"Alvaro D, Lisby M, Rothstein R",18085829,PMC2134942,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,cell,Homo sapiens,high content screen,secondary screen,RNAi screen,fluorescence microscopy,Comparative RNAi screening identifies a conser...,"Rohn JL, Sims D, Liu T, Fedorova M, Schöck F, ...",21893601,PMC3171124,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,cell,Homo sapiens,high content screen,primary screen,RNAi screen,fluorescence microscopy,Genome-wide RNAi screening identifies human pr...,"Simpson JC, Joggerst B, Laketa V, Verissimo F,...",22660414,PMC3171124,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,cell,Homo sapiens,high content screen,primary screen,protein screen,fluorescence microscopy,Whole-genome screening identifies proteins loc...,"Fong KW, Li Y, Wang W, Ma W, Li K, Qi RZ, Liu ...",24127217,PMC3798253,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,cell,Homo sapiens,high content screen,primary screen,protein screen,fluorescence microscopy,Whole-genome screening identifies proteins loc...,"Fong KW, Li Y, Wang W, Ma W, Li K, Qi RZ, Liu ...",24127217,PMC3798253,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Pubtator 

 This section uses the input file "pubmedid" that contains PubMed IDs to get annotations from https://www.ncbi.nlm.nih.gov/research/pubtator/

In [6]:

final = []
def SubmitPMIDList(Inputfile, Format, Bioconcept):
    json = {}
    #
    # load pmids
    #
    with io.open(Inputfile, 'r', encoding="utf-8") as file_input:
        json = {"pmids": [pmid.strip() for pmid in file_input.readlines()]}
        ids1 = (json["pmids"])
        
    #
    # load bioconcepts
    #
    if Bioconcept != "":
        json["concepts"] = Bioconcept.split(",")

    #
    # request
    #
    r = requests.post("https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/" + Format, json=json)
    if r.status_code != 200:
        print("[Error]: HTTP code " + str(r.status_code))
    else:
        full_text=r.text
        #print(full_text)
        final.append(full_text)
        

        # Writing to pubtator.json
        with open("pubtator.json", "w") as outfile:
            
            outfile.write(full_text)
            print("done")
            
SubmitPMIDList("PubMedID.pmid", "biocjson","dieases")

done


In [8]:
List1 = []
with open('pubtator.json') as f:
    for jsonObj in f:
        PubTatorDict = json.loads(jsonObj)
        List1.append(PubTatorDict)

In [9]:
List1

[{'_id': '34595747|None',
  'id': '34595747',
  'infons': {},
  'passages': [{'infons': {'journal': 'Neuropathol Appl Neurobiol;2021Sep30. doi:10.1111/nan.12770',
     'year': '2021',
     'type': 'title',
     'authors': 'Verzat C, Harley J, Patani R, Luisier R, ',
     'section': 'Title'},
    'offset': 0,
    'text': 'Image-based deep learning reveals the responses of human motor neurons to stress and VCP-related ALS.',
    'sentences': [],
    'annotations': [{'id': '2',
      'infons': {'identifier': '9606', 'type': 'Species'},
      'text': 'human',
      'locations': [{'offset': 51, 'length': 5}]},
     {'id': '3',
      'infons': {'identifier': '7415',
       'type': 'Gene',
       'ncbi_homologene': '5168'},
      'text': 'VCP',
      'locations': [{'offset': 85, 'length': 3}]}],
    'relations': []},
   {'infons': {'type': 'abstract', 'section': 'Abstract'},
    'offset': 102,
    'text': "OBJECTIVES: Although morphological attributes of cells and their substructures are reco

## Neo4j

### For IDR

In [11]:
for i in total: #idr
    session.run("""
            MERGE (a:id {name:$name})
            MERGE (b:organism {name:$name1})
            MERGE (c:pubmedID {name:$name2})
            MERGE (d:Screen_Technology_Type {name:$name3})
            MERGE (e:Study_Type {name:$name4})
            MERGE (f:PMCID {name:$name5})
            MERGE (g:Screen_Type {name:$name6})
            MERGE (h:Imaging_Method {name:$name7})
            MERGE (l:Publication_Title {name:$name11})
            MERGE (m:Publication_Author {name:$name12})
            MERGE (n:Publication_DOI {name:$name13})
            
            MERGE (a)-[:isOrganism]->(b) 
            MERGE (a)-[:isID]->(c) 
            MERGE (a)-[:isScreenType]->(d)
            MERGE (a)-[:isStudyType]->(e) 
            MERGE (a)-[:isPMCID]->(f) 
            MERGE (a)-[:isScreenType]->(g)
            MERGE (a)-[:isImaging]->(h)
            MERGE (a)-[:isID]->(l) 
            MERGE (a)-[:isAuthor]->(m)
            MERGE (a)-[:isDoi]->(n)

            """ ,name= i["Screen_Id"],name1= i["Organism"],name2= i["PubMed ID"], 
                        name3 =i["Screen Technology Type"], name4=i["Study Type"], 
                        name5=i["PMC ID"], name6=i["Screen Type"], name7=i["Imaging Method"],
                        name11= i["Publication Title"]
                        ,name12= i["Publication Authors"],name13= i["Publication DOI"])

## For Pubtator

In [12]:
for j in List1: #pubtator

        session.run("""MERGE (i:pubid {name:$name8})
            MERGE (j:year {name:$name9})
            MERGE (k:author {name:$name10})

            MERGE (i)-[:isyear]->(j)
            MERGE (k)-[:isAuthor]->(j)
            MERGE (i)-[:isID]->(k)


            """,name8 = j["id"], name9= j["year"], name10 = j["authors"])

## Merge the data from pubtator and idr

In [13]:
session.run("""match(n:pubmedID) match(p:pubid)
        where n.name=p.name
        create(n)-[:connected_graph]->(p)
        """)

## Link Prediction

### This cypher queries are run in the Neo4j browser

## Common Neighbors

match(n:pubmedID) match(p:pubid)

RETURN gds.alpha.linkprediction.commonNeighbors(n, p) AS score

## Adamic Adar

match(n:pubmedID) match(p:pubid)

RETURN gds.alpha.linkprediction.adamicAdar(n, p) AS score

## Total Neighbor

match(n:pubmedID) match(p:pubid)

RETURN gds.alpha.linkprediction.totalNeighbors(n, p) AS score

## Same Community

match(n:pubmedID) match(p:pubid)

RETURN gds.alpha.linkprediction.sameCommunity(n, p) AS score

## Preferential Attachment

match(n:pubmedID) match(p:organism)

RETURN gds.alpha.linkprediction.preferentialAttachment(n, p) AS score

## Resource Allocation

match(n:pubmedID) match(p:organism)

RETURN gds.alpha.linkprediction.resourceAllocation(n, p) AS score

## Semantic Queries

## Q1. How can I find image data with a given source, comparability, attributes, etc?

 MATCH p=()-[r:isImaging]->() RETURN p


## Q2. What is the shortest path between {Entity1} and {Entity2} and what is this path?


MATCH (n:pubmedID{name:"25373780"}), (m:organism {name:"Homo sapiens"}), p= shortestPath((n)-[*]-(m)) WHERE length(p) > 1 

RETURN p


## Q3. How can I search for a specific ID?

 match(n:pubmedID{name:"25373780"}) Return n


## Q4. How can I get all relationships for a node with cypher?

 MATCH (:organism{name:"Homo sapiens"})-[r]-() RETURN r


## Q5. How far are {Document1} and {Document2} apart?	

MATCH (n:pubmedID{name:"25373780"}), (m:pubmedID {name:"24943848"}), p= shortestPath((n)-[*]-(m)) WHERE length(p) > 1 RETURN p


## Q6. How can you group and count relationship types?

MATCH (n {name: '25373780'})-[r]->()
RETURN type(r), count(*)
